In [11]:
from logger import Logger
from preparator import Preparator 
from solver import Solver 
import pandas as pd 
from tqdm import tqdm 
from datetime import datetime

import pennylane as qml 
from pennylane import numpy as np
testdev = qml.device('default.qubit', wires = 1)

In [12]:
size = 10
shots = 1000
matrix_number = 60
source_name = 'Training_size_{size}.csv'
ansatz = 'CIA'
template = pd.read_csv('data/template.csv')

In [13]:
# Prepare train datasets
log = Logger (log_template = template, log_path = f'data/train_{ansatz}/{source_name.format(size = size)}')
exists = log.create_logfile()

Logfile data/train_CIA/Training_size_10.csv already exists


In [14]:
log_file, id = log._load_logfile()

prep = Preparator()

# for i in tqdm(range(matrix_number), desc = f"Generating random matrices with size {size}"):
    
#     id += 1
#     matrix = prep.rankPreparator(size = size, rank = size)
#     current_datetime = datetime.now()
#     data = { 
#         'Id (int)' : id, 
#         'Valid (int)' : 1, 
#         'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
#         'Matrix (mat)' : matrix,
#         'Size (int)' : size,
#         'Type (str)' : 'Random',
#         'Rank (int)' : size,
#         'Density (flt)' : -1.0,
#         'Stiffness (flt)' : -1.0,
#         'Other (flt)' : -1.0,
#         'Test_solver (str)' : '-',
#         'Test (flt)' : -1.0,
#         'Test_state (vec)' : [],
#         'Test_iterations (int)' : -1,
#         'Test_time (flt)' : -1.0,
#         'Train_solver (str)' : '-',
#         'Preprocessing (str)' : '-',
#         'Postprocessing (str)' : '-',
#         'Train (flt)' : -1.0,
#         'Train_state (vec)' : [],
#         'Train_iterations_1 (int)' : -1,
#         'Train_iterations_2 (int)' : -1,
#         'Train_iterations_3 (int)' : -1,
#         'Train_shots (int)': -1, 
#         'Train_cost (flt)' : -1.0,
#         'Train_time (flt)' : -1.0, 
#         'Flags (str)' : '',
#         'Baren (int)' : -1
#     }
    
#     log.log(data)



Columns in data/train_CIA/Training_size_10.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other (flt)', 'Test_solver (str)', 'Test (flt)',
       'Test_state (vec)', 'Test_iterations (in

In [15]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data

In [16]:

ids = range (1,61)
number_of_solvers = 30
poolsize = len(ids)//number_of_solvers

In [17]:

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)

    args = []
    for i in range (len(pool_ids)):
        if (size > 11):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(number_of_solvers) as pool:
        processed_data = pool.starmap(test, args)


for i in range(number_of_solvers):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 


tensor([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12],
        [13, 14],
        [15, 16],
        [17, 18],
        [19, 20],
        [21, 22],
        [23, 24],
        [25, 26],
        [27, 28],
        [29, 30],
        [31, 32],
        [33, 34],
        [35, 36],
        [37, 38],
        [39, 40],
        [41, 42],
        [43, 44],
        [45, 46],
        [47, 48],
        [49, 50],
        [51, 52],
        [53, 54],
        [55, 56],
        [57, 58],
        [59, 60]], requires_grad=True)

Columns in data/train_CIA/Training_size_10.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other (flt)', 'Test_solver (str)', 'Test (flt)',
       'Test_state (vec)', 'Test_iterations (in

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, lis

Columns in data/train_CIA/Training_size_10.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other (flt)', 'Test_solver (str)', 'Test (flt)',
       'Test_state (vec)', 'Test_iterations (in

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, lis

...

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, lis

Columns in data/train_CIA/Training_size_10.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other (flt)', 'Test_solver (str)', 'Test (flt)',
       'Test_state (vec)', 'Test_iterations (in

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, lis

Columns in data/train_CIA/Training_size_10.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other (flt)', 'Test_solver (str)', 'Test (flt)',
       'Test_state (vec)', 'Test_iterations (in

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, lis

In [18]:
display(processed_data)

[[{'Id (int)': 1,
   'Valid (int)': 1,
   'Updated (str)': '2025-08-24 20:28:30',
   'Matrix (mat)': array([[ 2.44299934e-01, -2.45526759e-01,  1.90816810e-01,
            3.28883498e-02,  1.31116447e-01,  2.16677809e-02,
            3.54943402e-01,  1.26995007e-01, -7.10919607e-02,
           -1.48540943e-01],
          [-2.45526759e-01, -3.23149038e-01,  2.60379604e-02,
           -2.50670288e-01, -1.54003226e-01, -2.08378166e-01,
           -1.19471270e-01, -3.18624954e-01, -6.40183843e-01,
           -1.70439716e-01],
          [ 1.90816810e-01,  2.60379604e-02,  6.63255576e-02,
            1.09787332e-01,  1.13105186e-01,  8.80375344e-02,
            1.84815765e-01,  1.69560915e-01,  2.15941007e-01,
            4.05919486e-03],
          [ 3.28883498e-02, -2.50670288e-01,  1.09787332e-01,
           -7.80852249e-02,  6.61130819e-03, -6.83239834e-02,
            1.39865236e-01, -5.44066556e-02, -2.81654396e-01,
           -1.42297040e-01],
          [ 1.31116447e-01, -1.54003226e-0

In [10]:
display(len(processed_data))

10

In [21]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False)

        trained_data['Train (flt)'] = train
        trained_data['Train_state (vec)'] = train_bitstring
        trained_data['Train_iterations (int)'] = train_iterations
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

        
        

In [22]:

from multiprocessing import Pool
import pandas as pd 



stepsizes = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0]

cores = 20

ids = np.arange(1,61,1)
ids = ids.reshape(cores, len(ids) // cores)


print(ids)
solvers = [] 

depth = 1

template = pd.read_csv('data/template_train.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = size, shots = 1000)
    solvers.append(Solver(dev = dev, ansatz = ansatz, gate = 'X', iterations_limit = 1000, accepterr = [0.01, 0.05, 0.1], baren_check = True, baren_rerr = 0.01))


for stepsize in stepsizes: 
    print(f'Started computing Training set for stepsize = {stepsize}')
    log = Logger(log_template = template, log_path = f'data/train_{ansatz}/Training_size_{size}_stepsize_{stepsize}.csv')
    train_dataset, _ = log._load_logfile()

    data = [[log.read(id) for id in subset] for subset in ids]
    #data = [[log.read(id) for id in range (id_start, id_start + len(train_dataset) // cores)] for id_start in range (1, len(train_dataset), len(train_dataset) // cores) ]
    
    #print(data)
    if (__name__ == '__main__') and (len(train_dataset) % cores):
        
        args = [(data[i], solvers[i], stepsize, depth) for i in range (cores)]
        #print(train_dataset)
        #print(args)
        with Pool(cores) as pool:
            processed_data = pool.starmap(train, args)


    for i in range(cores):
        for data in processed_data[i]:
            log.log(data = data, byid = True, Id = data['Id (int)'])
    


[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]
 [16 17 18]
 [19 20 21]
 [22 23 24]
 [25 26 27]
 [28 29 30]
 [31 32 33]
 [34 35 36]
 [37 38 39]
 [40 41 42]
 [43 44 45]
 [46 47 48]
 [49 50 51]
 [52 53 54]
 [55 56 57]
 [58 59 60]]
Started computing Training set for stepsize = 0.01


/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, 

Started computing Training set for stepsize = 0.05


/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, 

Started computing Training set for stepsize = 0.1


...

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, 

Started computing Training set for stepsize = 1.5


/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, 

Started computing Training set for stepsize = 2.0


/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, 

In [26]:
display(processed_data[0])

{'Id (int)': 1,
 'Valid (int)': 1,
 'Updated (str)': '2025-08-21 01:28:00',
 'Matrix (mat)': array([[ 2.44299934e-01, -2.45526759e-01,  1.90816810e-01,
          3.28883498e-02,  1.31116447e-01,  2.16677809e-02,
          3.54943402e-01,  1.26995007e-01, -7.10919607e-02,
         -1.48540943e-01],
        [-2.45526759e-01, -3.23149038e-01,  2.60379604e-02,
         -2.50670288e-01, -1.54003226e-01, -2.08378166e-01,
         -1.19471270e-01, -3.18624954e-01, -6.40183843e-01,
         -1.70439716e-01],
        [ 1.90816810e-01,  2.60379604e-02,  6.63255576e-02,
          1.09787332e-01,  1.13105186e-01,  8.80375344e-02,
          1.84815765e-01,  1.69560915e-01,  2.15941007e-01,
          4.05919486e-03],
        [ 3.28883498e-02, -2.50670288e-01,  1.09787332e-01,
         -7.80852249e-02,  6.61130819e-03, -6.83239834e-02,
          1.39865236e-01, -5.44066556e-02, -2.81654396e-01,
         -1.42297040e-01],
        [ 1.31116447e-01, -1.54003226e-01,  1.13105186e-01,
          6.61130819

In [38]:
for i in range(cores):
    data = processed_data[i]
    log.log(data = data, byid = True, Id = data['Id (int)'])

In [36]:
cores = 30
for i in range(cores):
        for data in processed_data[i]:
            log.log(data = data, byid = True, Id = data['Id (int)'])